In [2]:
import pandas as pd
import numpy as np
import scipy as sc
import scipy.sparse as sp
import pylab 
import sys
import time
import os
import utils_my
from utils_my import *
from sklearn.externals.joblib import dump, load, Parallel, delayed
import xgboost as xgb

In [3]:
rseed = 0
xgb_eta = .3
n_passes = 5
n_trees = 40
n_iter = 7
n_threads = 8
nr_factor = 4
learning_rate = .1
test_day = 30

In [4]:
"""
what's the t0tv_mx:
t0tv_mx is a matrix
'C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'banner_pos', 'device_type', 'device_conn_type'
count features
diffs features
exptv features in both unvariate and multivariate
"""
t0tv_mx_save = load("tiny_data/t0tv_mx.joblib_dat")
t0tv_mx = t0tv_mx_save['t0tv_mx']
click_values = t0tv_mx_save['click']
day_values = t0tv_mx_save['day']
#均匀分布
np.random.seed(rseed)
nn = t0tv_mx.shape[0]
r1 = np.random.uniform(0, 1, nn)

#22->29的四分之一
filter1 = np.logical_and(np.logical_and(day_values >= 22, day_values < test_day), np.logical_and(r1 < 0.25, True))
#30
filter_v1 = day_values == test_day
#22->29均匀取四分之一
xt1 = t0tv_mx[filter1, :]
yt1 = click_values[filter1]
if xt1.shape[0] <=0 or xt1.shape[0] != yt1.shape[0]:
    print(xt1.shape, yt1.shape)
    raise ValueError('wrong shape!')


In [13]:
dtrain = xgb.DMatrix(xt1,label=yt1)
dvalid = xgb.DMatrix(t0tv_mx[filter_v1],label=click_values[filter_v1])
watchlist = [(dtrain,'train'),(dvalid,'valid')]
print(xt1.shape, yt1.shape)
param = {'max_depth':6,'eta':.5,'objective':'binary:logistic','verbose':0,
'subsample':1.0, 'min_child_weight':50, 'gamma':0,
'colsample_bytree':.5, 'base_score':0.16, 'seed': rseed,'eval_metric': 'logloss'}

xgb_test_basis_d6 = xgb.train(param,dtrain,n_trees,watchlist)

dtv = xgb.DMatrix(t0tv_mx)
xgb_leaves = xgb_test_basis_d6.predict(dtv,pred_leaf=True)
print(xgb_leaves.shape)

(803360, 24) (803360,)
[15:49:18] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[0]	train-logloss:0.432767	valid-logloss:0.437081
[15:49:18] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[1]	train-logloss:0.425844	valid-logloss:0.435514
[15:49:18] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=6
[2]	train-logloss:0.422601	valid-logloss:0.433663
[15:49:18] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[3]	train-logloss:0.420677	valid-logloss:0.43394
[15:49:19] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[4]	train-logloss:0.418735	valid-logloss:0.434322
[15:49:19] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree 

In [17]:
for i in range(n_trees):
    pred2 = xgb_leaves[:, i]
    print(i, np.unique(pred2).size)
    t0['xgb_basis'+str(i)] = pred2

0 60
1 50
2 42
3 47
4 52
5 54
6 55
7 54
8 50
9 53
10 50
11 44
12 25
13 46
14 43
15 51
16 28
17 48
18 16
19 49
20 32
21 12
22 49
23 60
24 47
25 20
26 48
27 27
28 50
29 46
30 49
31 37
32 10
33 48
34 43
35 40
36 46
37 17
38 11
39 26


In [18]:
t0.head()

,click,xgb_basis0,xgb_basis1,xgb_basis2,xgb_basis3,xgb_basis4,xgb_basis5,xgb_basis6,xgb_basis7,xgb_basis8,...,xgb_basis30,xgb_basis31,xgb_basis32,xgb_basis33,xgb_basis34,xgb_basis35,xgb_basis36,xgb_basis37,xgb_basis38,xgb_basis39
0,0,104,61,51,57,77,92,85,101,74,...,96,48,13,54,49,50,81,30,19,43
1,0,104,59,51,63,68,78,61,101,67,...,95,48,13,53,49,50,63,30,19,43
2,0,104,59,51,58,77,92,66,101,65,...,96,48,13,63,45,50,81,30,19,43
3,1,94,80,68,63,65,73,82,93,69,...,4,48,13,68,77,50,63,24,20,43
4,1,88,59,68,63,70,92,61,101,65,...,84,48,13,53,49,50,63,30,19,43


In [19]:
fm_path="fm"
param_names = '_r' + str(rseed)
 
fn_t = "data/" + 'fm_' + param_names + '_t.txt'
fn_v = "data/" + 'fm_' + param_names + '_v.txt'

fm_cmd = fm_path + ' -k ' + str(nr_factor) + ' -t ' + str(n_iter) + ' -s '+ str(n_threads) + ' '
fm_cmd += ' -d ' + str(rseed) + ' -r ' + str(learning_rate) + ' ' + fn_v + ' ' + fn_t


In [21]:
print(fm_path)
print(param_names)
print(fn_t)
print(fn_v)
print(fm_cmd)

fm
_r0
data/fm__r0_t.txt
data/fm__r0_v.txt
fm -k 4 -t 7 -s 8  -d 0 -r 0.1 data/fm__r0_v.txt data/fm__r0_t.txt


In [28]:
t3a_save = load('t3a.joblib_dat')
t3a = t3a_save['t3a']

idx_base = 0
for vn in ['xgb_basis' + str(i) for i in range(n_trees)]:
    _cat = np.asarray(t0[vn].astype('category').values.codes, dtype='int32')
    _cat1 = _cat + idx_base
    print(vn, idx_base, _cat1.min(), _cat1.max(), np.unique(_cat).size)
    t3a[vn] = _cat1
    idx_base += _cat.max() + 1



t3a.ix[np.logical_and(np.logical_and(day_values < test_day, day_values >= 22), True),:].to_csv(open("tiny_data/fm_r0_t.txt", 'w'), sep='\t', header=False, index=False)
t3a.ix[day_values==test_day,:].to_csv(open("tiny_data/fm_r0_v.txt", 'w'), sep='\t', header=False, index=False)


xgb_basis0 0 0 59 60
xgb_basis1 60 60 109 50
xgb_basis2 110 110 151 42
xgb_basis3 152 152 198 47
xgb_basis4 199 199 250 52
xgb_basis5 251 251 304 54
xgb_basis6 305 305 359 55
xgb_basis7 360 360 413 54
xgb_basis8 414 414 463 50
xgb_basis9 464 464 516 53
xgb_basis10 517 517 566 50
xgb_basis11 567 567 610 44
xgb_basis12 611 611 635 25
xgb_basis13 636 636 681 46
xgb_basis14 682 682 724 43
xgb_basis15 725 725 775 51
xgb_basis16 776 776 803 28
xgb_basis17 804 804 851 48
xgb_basis18 852 852 867 16
xgb_basis19 868 868 916 49
xgb_basis20 917 917 948 32
xgb_basis21 949 949 960 12
xgb_basis22 961 961 1009 49
xgb_basis23 1010 1010 1069 60
xgb_basis24 1070 1070 1116 47
xgb_basis25 1117 1117 1136 20
xgb_basis26 1137 1137 1184 48
xgb_basis27 1185 1185 1211 27
xgb_basis28 1212 1212 1261 50
xgb_basis29 1262 1262 1307 46
xgb_basis30 1308 1308 1356 49
xgb_basis31 1357 1357 1393 37
xgb_basis32 1394 1394 1403 10
xgb_basis33 1404 1404 1451 48
xgb_basis34 1452 1452 1494 43
xgb_basis35 1495 1495 1534 40
xgb_b

C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [31]:
with open("tiny_data/fm_r0_t.txt") as f:
    buff = f.readline()
    a  = list(buff)

['0',
 '\t',
 '3',
 '0',
 '0',
 '0',
 '\t',
 '3',
 '0',
 '0',
 '2',
 '\t',
 '3',
 '0',
 '1',
 '0',
 '\t',
 '3',
 '0',
 '1',
 '9',
 '\t',
 '3',
 '0',
 '2',
 '6',
 '\t',
 '3',
 '2',
 '4',
 '3',
 '\t',
 '3',
 '4',
 '6',
 '0',
 '\t',
 '3',
 '4',
 '6',
 '3',
 '\t',
 '3',
 '5',
 '8',
 '1',
 '\t',
 '3',
 '7',
 '2',
 '0',
 '\t',
 '3',
 '7',
 '5',
 '5',
 '\t',
 '3',
 '9',
 '4',
 '7',
 '\t',
 '1',
 '2',
 '2',
 '7',
 '5',
 '\t',
 '2',
 '0',
 '7',
 '6',
 '1',
 '\t',
 '2',
 '9',
 '1',
 '0',
 '9',
 '\t',
 '3',
 '9',
 '4',
 '8',
 '3',
 '\t',
 '4',
 '9',
 '7',
 '6',
 '6',
 '\t',
 '1',
 '0',
 '6',
 '9',
 '4',
 '4',
 '\t',
 '1',
 '2',
 '3',
 '6',
 '3',
 '3',
 '\t',
 '1',
 '5',
 '7',
 '6',
 '8',
 '9',
 '\t',
 '2',
 '0',
 '4',
 '0',
 '1',
 '6',
 '\t',
 '2',
 '3',
 '9',
 '6',
 '0',
 '6',
 '\t',
 '2',
 '4',
 '4',
 '6',
 '5',
 '3',
 '\t',
 '2',
 '4',
 '4',
 '6',
 '5',
 '8',
 '\t',
 '2',
 '4',
 '4',
 '8',
 '3',
 '0',
 '\t',
 '2',
 '5',
 '6',
 '3',
 '8',
 '4',
 '\t',
 '2',
 '5',
 '7',
 '5',
 '3',
 '2',
 '\t',
